<center>
    <img src="https://github.com/juanandres-montero/macroeconometria/blob/main/logo%20EEC%20grande.png?raw=1" width="396" height="161"><br>
    <b style="font-size:200%">EC4301 MACROECONOMETRÍA</b><br>
    <b style="font-size:100%">Asistente: Juan Andrés Montero</b><br><br>
    <b style="font-size:100%"> Laboratorio #8:</b>
    <br>    
    <div style="font-size:150%; color:white; background-color: #0064b0; padding-top: 20px; padding-bottom: 20px; width: 50%; margin: 0 auto; border-radius: 15px; text-align: center;">
        Modelos ARCH y GARCH
    </div>
</center>

<br>

<i style="font-size:80%"> 
    Creado: Octubre 2024
</i>

<hr>

# Resumen del laboratorio anterior
## **Modelo ARIMA y Pronóstico**
- Utilizamos la metodología Box-Jenkins para ajustar un modelo ARIMA al IPC.
- Evaluamos la estabilidad del modelo y sus residuos.
- Realizamos pronósticos y evaluamos su precisión usando varias métricas.

# Simulación de Modelos ARCH y GARCH

En este laboratorio exploraremos la simulación y estimación de modelos ARCH y GARCH usando Stata. El código que veremos puede ser ejecutado en Stata y analizaremos los resultados.

## **1. Simulación de un ARCH(1)**

El modelo ARCH(1) que simularemos tiene la siguiente especificación:

- $y_{t} = 10 + \epsilon_{t}$
- $\epsilon_{t} = u_{t}\sqrt{0.4 + 0.5(\epsilon^2_{t-1})}$

donde u_{t} sigue una distribución normal estándar N(0,1).

## **2. Simulación de un GARCH(1,1)**

El modelo GARCH(1,1) que simularemos tiene la siguiente especificación:

- $y_{t} = 10 + \epsilon_{t}$
- $\epsilon_{t} = (u_{t})*\sigma_{t}$
- $\sigma^2_{t} =  0.2 + 0.4(\epsilon^2_{t-1}) + 0.6(\sigma^2_{t-1})$

donde u_{t} sigue una distribución normal estándar N(0,1).

# Aplicación Empírica: Análisis del Tipo de Cambio

## **Datos y Preparación**
Analizaremos la tasa de cambio porcentual del tipo de cambio de compra colón-dólar.

```stata
clear all
import excel "\tipo_cambio.xlsx", sheet("Hoja1") firstrow

* Declaramos nuestra variable temporal
tsset FECHA

* Calculamos la tasa de cambio porcentual de COMPRA
generate tasa_cambio = (D.COMPRA/L.COMPRA)*100
```

## **Análisis de Heteroscedasticidad**

1. Estimamos un modelo ARIMA inicial:
```stata
arima tasa_cambio, arima(4,0,4)
predict error, resid
```

2. Analizamos los residuos al cuadrado:
```stata
generate error_cuadrado = error*error
ac error_cuadrado
wntestq error_cuadrado
```

3. Test de McLeod-Li:
```stata
regress error_cuadrado L(1/15).error_cuadrado
```

## **Selección del Mejor Modelo**

Realizamos una búsqueda de cuadrícula sobre diferentes especificaciones GARCH:

```stata
forvalues p=0/2{
forvalues q=1/2{
    if `p'==0{
        quietly arch tasa_cambio, arch(1/`q') ar(4) ma(4)
    }
    else {
        quietly arch tasa_cambio, arch(1/`q') garch(1/`p') ar(4) ma(4)
    }
    quietly estat ic
    matrix temp = r(S)
    display "p = " `p' " q = " `q' " AIC = " temp[1,5] " BIC = " temp[1,6]
}
}
```